In [1]:
import pandas as pd
import csv

INPUT = "population_projections_Wales.csv"
OUTPUT = "wales_population_projections_tidy.csv"

rows = []

current_sex = None
reading_table = False
header = None

with open(INPUT, newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    for line in reader:
        # Skip empty lines
        if not line or all(x.strip() == "" for x in line):
            continue

        # --- Metadata: Gender line ---
        if line[0].startswith("Gender"):
            current_sex = line[1].strip().lower()
            continue

        # --- Column header for the table ---
        if line[0].strip() == "Age" and len(line) >= 3:
            header = [x.strip() for x in line]  # Age, 2022, 2030, ...
            reading_table = True
            continue

        # Stop when next block begins
        if line[0].startswith("National population projections"):
            reading_table = False
            header = None
            continue

        # --- Data rows ---
        if reading_table and header:
            age_label = line[0].strip()

            # protect against malformed / short rows
            if len(line) < len(header):
                continue

            # years start from header[1:]
            for col_idx in range(1, len(header)):
                year_str = header[col_idx].strip()
                if year_str == "":
                    continue
                year = int(year_str)

                pop = line[col_idx].strip()
                if pop == "":
                    continue

                rows.append({
                    "country": "Wales",
                    "sex": current_sex,
                    "age_group": age_label,
                    "year": year,
                    "population": int(pop)
                })

df = pd.DataFrame(rows)

# Clean age labels to match your standard
age_map = {
    "All Ages": "all",
    "Age 0 to 15": "0_15",
    "Age 16 to 24": "16_24",
    "Age 25 to 49": "25_49",
    "Age 50 to 64": "50_64",
    "Age 65+": "65+",
}
df["age_group"] = df["age_group"].map(age_map)

# Basic sanity checks
if df["sex"].isna().any():
    raise RuntimeError("Some rows have missing sex; check parsing.")
if df["age_group"].isna().any():
    bad = df.loc[df["age_group"].isna(), "age_group"].head(20)
    raise RuntimeError(f"Unmapped age labels found (showing up to 20):\n{bad}")

# Save
df.to_csv(OUTPUT, index=False)
print(f"Saved tidy Wales projections to {OUTPUT}")
print(df.head())

Saved tidy Wales projections to wales_population_projections_tidy.csv
  country    sex age_group  year  population
0   Wales  total       all  2022     3132676
1   Wales  total       all  2030     3291878
2   Wales  total       all  2035     3348642
3   Wales  total       all  2040     3396446
4   Wales  total       all  2045     3439378
